In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [42]:
def load_video(path, ms=100):
    
    # Video Capture 객체 생성
    capture = cv2.VideoCapture(path)
    
    # Frame을 저장할 List 선언
    frames = []
    
    while capture.isOpened(): # Video Capture가 준비되었는지 확인
        
        run, frame = capture.read() # 다음 Frame 읽기
        # ROI 설정 (x, y, width, height)
        roi_x, roi_y, roi_w, roi_h = 0, 280, 852, 200
        
        # ROI 추출
        roi = frame[roi_y:roi_y + roi_h, roi_x:roi_x + roi_w]

        # 그레이스케일 변환
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        edges = cv2.Canny(blurred, 50, 150)
        # 원본 프레임에 엣지 결과를 덮어쓰기 (ROI 영역만)
        frame[roi_y:roi_y + roi_h, roi_x:roi_x + roi_w] = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
        
        
        if run: # Frame을 읽은 경우
            # 결과 출력
            cv2.imshow('Frame with ROI Edges', frame)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        else: # 재생이 완료되어 더 이상 Frame을 읽을 수 없는 경우
            break
        
        # Frame List에 추가
        frames.append(frame)

    capture.release() # Capture 자원 반납
    cv2.destroyAllWindows() # 창 제거
    
    return np.array(frames, dtype='uint8')

In [43]:
# Video가 저장된 경로 입력
PATH = r".\plank_sample2.mp4"
# PATH = r"C:\Users\admin\Desktop\KUDIP\Video Samples\Hand Video2.mov"
# PATH = r"C:\Users\hj\AICV\수업\디지털영상처리\Video Samples\highway.mov"
# PATH = r"C:\Users\hj\AICV\수업\디지털영상처리\Video Samples\earth.avi"

# Video 재생 및 반환 (Numpy Array)
video = load_video(PATH)

In [7]:
print(f"- Data Type: {type(video)}")
print(f"- Data Shape: {video.shape} *Frames x Height x Width x Channel")
print(f"- Maximum Intensity: {video.max()}")
print(f"- Minimum Intensity: {video.min()}")

- Data Type: <class 'numpy.ndarray'>
- Data Shape: (201, 480, 852, 3) *Frames x Height x Width x Channel
- Maximum Intensity: 255
- Minimum Intensity: 0


In [50]:
import cv2
import numpy as np

# 영상 읽기
cap = cv2.VideoCapture('plank_sample2.mp4')

# 타이머 설정
total_time = 0
start_time = cv2.getTickCount()
stop_timer = False

def get_centroid(contour):
    M = cv2.moments(contour)
    if M["m00"] != 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        return (cX, cY)
    else:
        return (0, 0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 그레이스케일 변환 및 가우시안 블러 적용
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Canny 엣지 검출
    edges = cv2.Canny(blurred, 50, 150)

    # 객체 추출
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 상체와 하체의 중심점 초기화
    upper_body_centroid = (0, 0)
    lower_body_centroid = (0, 0)

    # 컨투어 처리
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 900:  # 노이즈 제거를 위한 최소 면적 설정
            x, y, w, h = cv2.boundingRect(contour)
            if y < frame.shape[0] // 2:  # 상체로 간주
                upper_body_centroid = get_centroid(contour)
                cv2.drawContours(frame, [contour], -1, (0, 255, 0), 2)
            else:  # 하체로 간주
                lower_body_centroid = get_centroid(contour)
                cv2.drawContours(frame, [contour], -1, (255, 0, 0), 2)

    # 중심점 그리기
    if upper_body_centroid != (0, 0):
        cv2.circle(frame, upper_body_centroid, 5, (0, 255, 0), -1)
    if lower_body_centroid != (0, 0):
        cv2.circle(frame, lower_body_centroid, 5, (255, 0, 0), -1)

    # 자세 체크
    if upper_body_centroid != (0, 0) and lower_body_centroid != (0, 0):
        distance = np.linalg.norm(np.array(upper_body_centroid) - np.array(lower_body_centroid))
        if distance > frame.shape[0] // 4:  # 예시 기준: 몸통 길이의 절반 이상 벗어나면 잘못된 자세로 간주
            stop_timer = True

    # 타이머 업데이트
    if not stop_timer:
        current_time = cv2.getTickCount()
        total_time = (current_time - start_time) / cv2.getTickFrequency()

    # 화면에 타이머 표시
    cv2.putText(frame, f'Time: {total_time:.2f} sec', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # 결과 출력
    cv2.imshow('Plank Pose Monitoring', frame)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
